# Lab 2 - QC-AFQMC with Amazon Braket and AWS Batch

## Introduction

In this lab, we will run a hybrid quantum-classical AFQMC workload for the hydrogen molecule using both Amazon Braket and AWS Batch. We use the [Amazon Braket Hybrid Job](https://docs.aws.amazon.com/braket/latest/developerguide/braket-what-is-hybrid-job.html) feature to perform shadow tomography. Then AWS Batch helps you to run the successive post-processing workloads using classical computing resources, in a similar way to our previous lab. 

In the following, we will go through the steps to
1. Create a hybrid job for shadow tomography using 4000 circuits (100 shots each)
2. Demonstrate how the post-processing of classical shadows are performed
3. Run a large scale QC-AFQMC example with 200 child jobs running on multiple compute instances in parallel
4. Compare the results of QC-AFQMC calculation with the previous classical AFQMC calculation

Let's create the boto3 clients for the services we will use in this lab. Some additional packages are needed for this section, so we'll install them first.

In [ ]:
pip install --quiet numba

In [ ]:
import boto3

sts_client = boto3.client("sts")
s3_client = boto3.client("s3")
cfn_client = boto3.client("cloudformation")
batch_client = boto3.client("batch")

In [ ]:
# Get my account ID
ACCOUNT_ID = sts_client.get_caller_identity().get("Account")
print("Account ID:", ACCOUNT_ID)

# Get my region 
my_session = boto3.session.Session()
WORKING_REGION = my_session.region_name
print("Region:", WORKING_REGION)

## Create a hybrid job

![](images/braket-hybrid-job.png)

Amazon Braket Hybrid Job offers a way for you to run hybrid quantum-classical algorithms requiring both classical AWS resources and quantum processing units (QPUs). Hybrid Jobs is ideal for long-running, iterative algorithms that involve the use of both classical computing resources and quantum computing resources. With Hybrid Jobs, after submitting your algorithm to run, Braket will run your algorithm in a scalable, containerized environment. Once the algorithm has completed, you can then retrieve the results.

Additionally, quantum tasks that are created from a hybrid job benefit from higher priority queueing to the target QPU device. This prioritization ensures that your quantum computations are processed and ran ahead of other tasks waiting in the queue. This is particularly advantageous for iterative hybrid algorithms, where the results of one quantum task depend on the outcomes of prior quantum tasks.

Specifically for our job, we will specify the following
- `device`: on which hardware the job is to be run
- `source_module`: folder of source code
- `entry_point`: which function is to be executed by the job
- `job_name`: job name
- `instance_config`: which type of EC2 instance is the classical part of the job hosted
- `output_data_config`: S3 bucket for storing the output of this job
- `hyperparameters`
  - `ShadowSize`: number of shadow circuits
  - `Shots`: number of shots for each circuit
  
Specifically, we include a [`requirements.txt`](./afqmc/requirements.txt) file inside the source folder, and the packages will be installed on the fly at the beginning of the job. In addition, you can also provide custom container image for your job.

In [ ]:
from braket.aws import AwsQuantumJob
from braket.jobs.config import InstanceConfig, OutputDataConfig
from braket.devices import Devices

data_bucket = f"s3://amazon-braket-batch-tutorial-{ACCOUNT_ID}-{WORKING_REGION}"
hybrid_job_output_prefix = "braket/lab-2-qc-afqmc-matchgate-circuits"

job = AwsQuantumJob.create(
    device="local:pennylane/lightning.qubit",
    source_module="afqmc",
    entry_point="afqmc.collect_shadow:main",
    job_name="lab-2-afqmc-matchgate-circuits",
    instance_config=InstanceConfig(instanceType="ml.m5.xlarge"),
    output_data_config=OutputDataConfig(
        s3Path=f"{data_bucket}/{hybrid_job_output_prefix}",
    ),
    hyperparameters={
        "ShadowSize": "4000",
        "Shots": "100",
    },
)

In [ ]:
job.state()

Once the hybrid job starts, it will change the status to `RUNNING`. We can also check the hybrid job status in the Braket console.

We can monitor the metrics in near-real time in the [Braket console page](https://console.aws.amazon.com/braket/) as shown below.

![](./images/hybrid-job-monitor.png)

<div class="alert alert-block alert-success">
<b>Activity:</b> Once your job is in the COMPLETED state, navigate to the S3 management console and to locate the results file.
</div>

In [ ]:
measurement_output = job.result()['output']
Q_save = job.result()['Q_save']

## Post-processing classical shadows

As we discussed in the introduction, the core quantities that need to be processed during the propagation of the walkers are the overlap $\langle\Psi_T|\phi\rangle$ and the local energy $\langle\Psi_T|H|\phi\rangle$.

Now let's take a closer look at how these classical shadows are post-processed to reconstruct these two quantities for the QC-QMC calculation. We choose an example walker state $|\phi_0\rangle$, which is the Hartree-Fock state. And we'll compute the overlap and local energy of the quantum trial state with $|\phi_0\rangle$. First let's find the electronic Hamiltonian for the hydrogen molecule (at equilibrium geometry).

In [ ]:
import time, copy
import numpy as np
import pennylane as qml

np.set_printoptions(precision=6, edgeitems=10, linewidth=150, suppress=True)

In [ ]:
###########################################################
# Prepare the necessary operators for AFQMC calculations. #
###########################################################
from pyscf.scf.hf import RHF
from pyscf.gto.mole import Mole
from pyscf import fci, gto, scf
from afqmc.utils.chemical_preparation import chemistry_preparation

# perform HF calculations
mol = gto.M(atom="H 0. 0. 0.; H 0. 0. 0.75", basis="sto-3g")
hf = mol.RHF()
hf.kernel()
prop = chemistry_preparation(mol, hf)

Angstrom_to_Bohr = 1.88973
symbols = ["H", "H"]
geometry = np.array([[0., 0., 0.], [0., 0., 0.75*Angstrom_to_Bohr]])
hamiltonian, _ = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=0, basis='sto-3g')

### Quantum trial state

Then let's take a look at the quantum trial state. We employed the [DoubleExcitation ansatz](https://docs.pennylane.ai/en/stable/code/api/pennylane.DoubleExcitation.html) from PennyLane in this tutorial, which is a widely used ansatz for solving the electronic structure problem. This ansatz is equivalent to the more famous [UCCSD ansatz](https://docs.pennylane.ai/en/stable/code/api/pennylane.UCCSD.html) in the case of the hydrogen molecule.

In [ ]:
# define the ansatz circuit
def V_T():
    qml.DoubleExcitation(0.12, wires=[0, 1, 2, 3])

# next we print out the statevector of this trial circuit operated on the Hartree Fock state
dev = qml.device("lightning.qubit", wires=4)
@qml.qnode(dev)
def hydrogen_circuit():
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    V_T()
    return qml.state()

state = hydrogen_circuit()
print(state)

As we can see, the quantum trial state has the following form:
$$
|\Psi_T\rangle = \alpha |1100\rangle + \beta |0011\rangle.
$$
The Hartree Fock state ($|\phi_0\rangle$) mentioned above can be represented as $|1100\rangle$.

<div class="alert alert-block alert-success">
<b>Activity:</b> Can you get the overlap $\langle\Psi_T|\phi_0\rangle$?
</div>

Next, we did some post-processing for the measured shadows.

In [ ]:
# then reconstruct the Q_list
def Q_reconstruct(signed_permutation):
    size = len(signed_permutation)
    Q = np.zeros((size, size))
    for i in range(size):
        Q[int(abs(signed_permutation[i])-1), i] = np.sign(signed_permutation[i])
    return Q

Q_list = []
for signed_permutation in Q_save:
    Q = Q_reconstruct(signed_permutation)
    Q_list.append(Q)

In [ ]:
from afqmc.utils.matchgate import construct_covariance

outcomes = []
for i in measurement_output:
    shadow_outcome = []
    for j in list(i.keys()):
        shadow_outcome.append([construct_covariance(j), i.get(j)])
    outcomes.append(shadow_outcome)

shadow = (outcomes, Q_list)

Finally we can construct a quantum trial state $|\Psi_T\rangle$ using the class `QTrial`, and we'll compute the overlap and local energy with $|\phi_0\rangle$.

In [ ]:
from afqmc.trial_wavefunction.quantum_ovlp import QTrial

phi0 = np.array([[1,0], [0,1], [0,0], [0,0]])
qtrial = QTrial(prop=prop, initial_state=[0, 1], ansatz_circuit=V_T, ifshadow=True, shadow=shadow)

In [ ]:
%time
# we'll first compute the overlap <\Psi_T|\phi>, the reference value have been provided
ovlp = qtrial.compute_ovlp(phi0)
print(f"The computed ovlp is {np.round(ovlp, 6)}, while the reference value is {0.998201}.")

In [ ]:
%time
# then we compute the local energy
local_energy, _ = qtrial.compute_local_energy(phi0, ovlp)
print(f"The computed local energy is {np.round(local_energy/ovlp + prop.nuclear_repulsion, 6)}, while the reference value is {-1.127071}.")

As you will see, the estimated overlap and local energy from classical shadows, are slightly off from the reference value. This is because shadow tomography is an approximation theory. The more shadows employed, the more accurate it will be.

## Large scale post-processing with Batch

So far, we have used Braket Hybrid Job to run matchgate circuits and collect measurement outcomes. Now we proceed to the classical post-processing step. We start with updaing the Batch job definition, since the post-processing of classical shadows requires more memory.

### Update job definition

<div class="alert alert-block alert-success">
<b>Activity:</b> Update the CloudFormation stack to increase the resource memory in the job definition to <b>2048</b>.
</div>

In [ ]:
with open('batch-environment.yaml', 'r') as file:
    template_body = file.read()

stack_name = 'batch-environment'
try:
    cfn_client.update_stack(
        StackName=stack_name,
        TemplateBody=template_body,
        Parameters=[
            {'ParameterKey': 'JobDefResourceMemory', 'ParameterValue': '2048'}
        ]
    )
    print("Waiting for CloudFormation stack to be updated...")
    waiter = cfn_client.get_waiter("stack_update_complete")
    waiter.wait(
        StackName=stack_name,
        WaiterConfig={
            'Delay': 10,
            'MaxAttempts': 150
        }
    )
    print("CloudFormation stack updated.")
except cfn_client.exceptions.ClientError as e:
    print(e)

print()
for output in cfn_client.describe_stacks(StackName=stack_name).get('Stacks')[0].get('Outputs'):
    print(f"{output.get('OutputKey')}: {output.get('OutputValue')}")
    if output.get('OutputKey') == "DataBucketName":
        data_bucket_name = output.get('OutputValue')
    if output.get('OutputKey') == "JobQueue":
        job_queue = output.get('OutputValue')
    if output.get('OutputKey') == "JobDefinition":
        job_definition = output.get('OutputValue')

### Run the Batch job

Next we submit the Batch job, just like we did in lab-1. Note that we have to specify one additiona environment variable `JOB_INPUT_FILE_KEY`, pointing to the S3 location where the classical shadows are saved.

In [ ]:
response = batch_client.submit_job(
    jobName="lab-2-quantum-large-scale",
    jobQueue=job_queue,
    jobDefinition=job_definition,
    containerOverrides={
        "environment": [
            {"name": "JOB_INPUT_FILE_KEY", "value": f"{hybrid_job_output_prefix}/output/model.tar.gz"},
            {"name": "JOB_ENTRY_POINT", "value": "run_qc_afqmc"},
            {"name": "JOB_DTAU", "value": "0.005"},
            {"name": "JOB_TIME_STEPS", "value": "200"}
        ]
    },
    arrayProperties={"size": 200}
)

jobId = response["jobId"]

In [ ]:
batch_client.describe_jobs(jobs=[jobId]).get("jobs")[0].get("status")

## Retrieve results of large-scale post-processing job

Finally, let's summarize the QC-AFQMC calculation results, and compare it against its classical counterpart.

In [ ]:
import json
import numpy as np
from pathlib import Path

local_energies_real = []
local_energies_imag = []
weights = []

for i in range(200):
    with open(f"results/lab-1/result_{i}.json", "r") as file:
        data = json.load(file)
    [local_energies_real.append(j) for j in data["local_energies_real"]]
    [local_energies_imag.append(j) for j in data["local_energies_imag"]]
    [weights.append(j) for j in data["weights"]]

local_energies = [[ii+1.j*jj for ii, jj in zip(i, j)] for i, j in zip(local_energies_real, local_energies_imag)]   
classical_energies = np.real(np.average(local_energies, weights=weights, axis=0))

In [ ]:
import json
import numpy as np
from pathlib import Path

job = batch_client.describe_jobs(jobs=[jobId])["jobs"][0]
job_status = job["status"]
if job_status == "SUCCEEDED":
    Path("results/lab-2").mkdir(parents=True, exist_ok=True)
    s3_client = boto3.client("s3")
    for i in range(200):
        s3_client.download_file(
            data_bucket_name,
            f"batch/{jobId}:{i}/results.json",
            f"results/lab-2/result_{i}.json"
        )
else:
    print(f"Your job is in status {job_status}")

local_energies_real = []
local_energies_imag = []
weights = []

for i in range(200):
    with open(f"results/lab-2/result_{i}.json", "r") as file:
        data = json.load(file)
    [local_energies_real.append(j) for j in data["local_energies_real"]]
    [local_energies_imag.append(j) for j in data["local_energies_imag"]]
    [weights.append(j) for j in data["weights"]]

local_energies = [[ii+1.j*jj for ii, jj in zip(i, j)] for i, j in zip(local_energies_real, local_energies_imag)]   
qc_energies = np.real(np.average(local_energies, weights=weights, axis=0))

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.plot(
    0.005 * np.arange(600),
    classical_energies,
    linestyle="dashed",
    marker=".",
    color="tab:blue",
    label="classical",
)
plt.plot(
    0.005 * np.arange(200),
    qc_energies,
    linestyle="dashed",
    marker=".",
    color="tab:orange",
    label="quantum",
)
plt.axhline(-1.137117067345732, linestyle="dashed", color="black")
plt.title(r"Ground state estimation of H$_2$ using AFQMC", fontsize=16)
plt.legend(fontsize=14, loc="upper right")
plt.xlabel(r"$\tau$", fontsize=14)
plt.ylabel("Energy", fontsize=14)
plt.yticks(fontsize=14)
plt.tick_params(direction="in", labelsize=14)
plt.show()

## Wrapping up

<div class="alert alert-block alert-info">
<b>You reached the end of lab 2. Well done!</b>
</div>